In [1]:
import requests
import json
import os
import sqlalchemy
import pandas as pd
import mdapi_functions as md
from IPython.display import display, Markdown
import markdown
from bokeh.plotting import figure, show
from bokeh.models import (Span, TabPanel, Tabs, ColumnDataSource, DataCube,
                          GroupingInfo, StringFormatter, SumAggregator,
                          TableColumn, HoverTool)
from bokeh.palettes import Category20c
from bokeh.transform import cumsum
from bokeh.io import output_notebook
from math import pi
from datetime import datetime
import datacite_api_functions as dcf
import numpy as np
import data_doc_helper as dh

from math import pi


In [2]:
output_notebook(hide_banner=True)

x = {
    'United States': 157,
    'United Kingdom': 93,
    'Japan': 89,
    'China': 63,
    'Germany': 44,
    'India': 42,
    'Italy': 40,
    'Australia': 35,
    'Brazil': 32,
    'France': 31,
    'Taiwan': 31,
    'Spain': 29,
}

data = pd.Series(x).reset_index(name='value').rename(columns={'index': 'country'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = Category20c[len(x)]

p = figure(height=350, title="Pie Chart", toolbar_location=None,
           tools="hover", tooltips="@country: @value", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='country', source=data)

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None

show(p)

In [3]:
output_notebook(hide_banner=True)

df = md.get_freeze_profile()
df = df[df["demo_grp"].isin(["ethnicity"])]
df2 = df[["cat", "n"]]
df2["angle"] = df2["n"]/ df2["n"].sum() * 2*pi
df2['color'] = Category20c[df2.__len__()]
df2["tot"] = df2["n"].sum()
df2

p = figure(height=350, title="Pie Chart", toolbar_location=None,
           tools="hover", tooltips="@cat: @n / @tot", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='cat', source=df2)

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None
show(p)


In [4]:
output_notebook(hide_banner=True)

df = md.get_freeze_profile()
df = df[df["demo_grp"].isin(["sex"])]
df2 = df[["cat", "n"]]
df2["angle"] = df2["n"]/ df2["n"].sum() * 2*pi
df2['color'] = Category20c[df2.__len__()]
df2["tot"] = df2["n"].sum()
df2

p = figure(height=350, title="Pie Chart", toolbar_location=None,
           tools="hover", tooltips="@cat: @n / @tot", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='cat', source=df2)

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None
show(p)


In [5]:
output_notebook(hide_banner=True)

df = md.get_freeze_profile()
df = df[df["demo_grp"].isin(["year_of_birth"])]
df2 = df[["cat", "n"]]
df2["angle"] = df2["n"]/ df2["n"].sum() * 2*pi
df2['color'] = Category20c[df2.__len__()]
df2["tot"] = df2["n"].sum()
df2

p = figure(height=350, title="Pie Chart", toolbar_location=None,
           tools="hover", tooltips="@cat: @n / @tot", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='cat', source=df2)

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None
show(p)


In [4]:
df = md.prep_dsvs_for_gb_pages()

In [8]:
df[df["source"] == "TEDS"]

,source,source_table,version_num,table,version_date,table_name,short_desc,long_desc,collection_start,collection_end,topic_tags,source_name,research_grouping,participants_invited,participants_included,num_rows,num_columns,Owner,geographic_coverage_Nations,geographic_coverage_Regions
381,TEDS,TEDS_TEDS_21yr_twin_covid_phase2,2,TEDS_21yr_twin_covid_phase2,20250206.0,Teds 21 Year Twin COVID-19 Phase 2,,Actively participating twins contactable by em...,06/2020,07/2020,,Twins Early Development Study (TEDS),"King's College London, Institute of Psychiatry...",8394,3706,3708.0,233,King's College London,"England, Wales",Nationwide
385,TEDS,TEDS_TEDS_21yr_twin_covid_phase4,2,TEDS_21yr_twin_covid_phase4,20250206.0,Teds 21 Year Twin COVID-19 Phase 4,,Actively participating twins contactable by em...,03/2021,04/2021,,Twins Early Development Study (TEDS),"King's College London, Institute of Psychiatry...",8394,3641,3648.0,227,King's College London,"England, Wales",Nationwide
383,TEDS,TEDS_TEDS_21yr_twin_covid_phase3,2,TEDS_21yr_twin_covid_phase3,20250206.0,Teds 21 Year Twin COVID-19 Phase 3,,Actively participating twins contactable by em...,10/2020,11/2020,,Twins Early Development Study (TEDS),"King's College London, Institute of Psychiatry...",8394,3349,3355.0,229,King's College London,"England, Wales",Nationwide
387,TEDS,TEDS_TEDS_21yr_twin_phase1_qnr,2,TEDS_21yr_twin_phase1_qnr,20250206.0,Teds 21 Year Twin Phase 1 Questionnaire,,All contactable participating twins in TEDS we...,06/2017,02/2019,,Twins Early Development Study (TEDS),"King's College London, Institute of Psychiatry...",20986,8757,8793.0,463,King's College London,"England, Wales",Nationwide
379,TEDS,TEDS_TEDS_21yr_twin_covid_phase1,2,TEDS_21yr_twin_covid_phase1,20250206.0,Teds 21 Year Twin COVID-19 Phase 1,,Participating twins contactable by email were ...,04/2020,05/2020,,Twins Early Development Study (TEDS),"King's College London, Institute of Psychiatry...",14619,4400,4408.0,230,King's College London,"England, Wales",Nationwide
393,TEDS,TEDS_TEDS_background,2,TEDS_background,20250206.0,TEDS Background,,All participating parents in TEDS were sent a ...,01/1995,11/2005,,Twins Early Development Study (TEDS),"King's College London, Institute of Psychiatry...",32606,10845,11018.0,47,King's College London,"England, Wales",Nationwide
391,TEDS,TEDS_TEDS_26yr_mhq,2,TEDS_26yr_mhq,20250206.0,Teds 26 Year Mental Health Questionnaire,,All contactable participating twins in TEDS we...,07/2021,11/2023,,Twins Early Development Study (TEDS),"King's College London, Institute of Psychiatry...",20962,7866,7878.0,798,King's College London,"England, Wales",Nationwide
389,TEDS,TEDS_TEDS_21yr_twin_phase2_qnr,2,TEDS_21yr_twin_phase2_qnr,20250206.0,Teds 21 Year Twin Phase 2 Questionnaire,,Contactable participating twins in TEDS were a...,02/2018,02/2019,,Twins Early Development Study (TEDS),"King's College London, Institute of Psychiatry...",17128,7492,7518.0,326,King's College London,"England, Wales",Nationwide


In [6]:
md.get_md_api_dsvs()

,source,index,values_table,description_table,version_date,num_rows,table,table_full,version_num,num_columns,num_participants
0,ALSPAC,0,None,None,20250307.0,5708.0,custom_llc0026_G1ethnicity,custom_llc0026_G1ethnicity_v0001_20250307,v0001,12,5708.0
1,ALSPAC,1,None,None,20250307.0,694.0,custom_llc0032_G0m,custom_llc0032_G0m_v0001_20250307,v0001,469,677.0
2,ALSPAC,2,None,None,20250307.0,5708.0,custom_llc0032_G1,custom_llc0032_G1_v0001_20250307,v0001,3420,5708.0
3,ALSPAC,3,custom_metabolomicsm_v0001_values_20220810,custom_metabolomicsm_v0001_description_20220810,20220810.0,1818.0,custom_metabolomicsm,custom_metabolomicsm_v0001_20220810,v0001,235,358.0
4,ALSPAC,4,custom_metabolomicsy_v0001_values_20220810,custom_metabolomicsy_v0001_description_20220810,20220810.0,2850.0,custom_metabolomicsy,custom_metabolomicsy_v0001_20220810,v0001,231,2830.0
...,...,...,...,...,...,...,...,...,...,...,...
738,nhsd,738,None,NPEX_v0002_description,NaN,2440208.0,NPEX,NPEX_v0002,v0002,38,NaN
739,nhsd,739,None,NPEX_v0003_description,NaN,2807397.0,NPEX,NPEX_v0003,v0003,38,NaN
740,nhsd,740,None,PCM_v0001_description,NaN,19239373.0,PCM,PCM_v0001,v0001,57,NaN
741,nhsd,741,None,PCM_v0002_description,NaN,22859974.0,PCM,PCM_v0002,v0002,57,NaN
